# Coordinate Generator

> Parte do Sistema responsável por gerar o vetor de coordenadas a partir dos tiles gerados pela rede neural.

## Imports

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np

from PIL import Image
import cv2

from os import listdir
from os.path import isfile, join

from math import pi
from numpy import arctan
from numpy import sinh
import random

from skimage.morphology import skeletonize, thin
from skimage import data
import matplotlib.pyplot as plt
from skimage.util import invert


## Conversão da posição de um pixel no tile para sua coordenada de latitude e longitude

In [2]:


def get_lat(x, y, z):
  n = 2 ** z

  min_rad = arctan(sinh(pi * (1 - 2 * y / n)))
  min_deg = min_rad * 180.0 / pi

  max_rad = arctan(sinh(pi * (1 - 2 * (y+1) / n)))
  max_deg = max_rad * 180.0 / pi

  return min_deg, max_deg


def get_lon(x, y, z):
  n = 2 ** z
  min_deg = x / n * 360.0 - 180.0
  max_deg = (x+1) / n * 360.0 - 180.0
  return min_deg, max_deg


def pixToCoord(x, y, z, t_height, t_width, px, py):

  [min_lat, max_lat] = get_lat(x,y,z)
  [min_lon, max_lon] = get_lon(x,y,z)
  
  lat = min_lat + py*((max_lat - min_lat)/t_height)
  lon = min_lon + px*((max_lon - min_lon)/t_width)
  
  coord = (lat, lon)
  

  return coord



## Função de busca em largura

In [3]:


def findLine(image):
    img = image
    maxSort = (256 * 256)

    *numSorts, = range(0, 256)
    random.shuffle(numSorts)
    *numSorts2, = range(0, 256)
    random.shuffle(numSorts2)

    
    for i in range(0, maxSort):

        if i > 256:
            *numSorts, = range(0, 256)
            random.shuffle(numSorts)
            *numSorts2, = range(0, 256)
            random.shuffle(numSorts2)
            
        
        pxSorted = numSorts[i % 256]
        pySorted = numSorts2[i % 256]
        
        i = pySorted
        j = pxSorted
        
        if img[i, j] == 0:
            img = colorFrontier(img, j, i, 1)

    img, line = getLines(img)
    
                
    return img, line



def colorFrontier(img, px, py , state):
    visitados = set()
    prox = []
    prox.append((px, py, state))
        
    while len(prox) != 0:
        
        px, py, state = prox.pop(0)

        atual = (px, py)
        if atual not in visitados:
            img[px, py] = state

            state = state + 1
            pxMin = px - 1
            pxMax = px + 1
            pyMin = py - 1
            pyMax = py + 1
            
            if(pxMin >= 0 and img[pxMin, py] == 0):
                    prox.append( (pxMin, py, state) )

            if(pxMax < img.shape[0] and img[pxMax, py] == 0):
                    prox.append( (pxMax, py, state) )

            if(pyMin >= 0 and img[px, pyMin] == 0):
                    prox.append( (px, pyMin, state) )

            if(pyMax < img.shape[1] and img[px, pyMax] == 0):
                    prox.append( (px, pyMax, state) )
                    
                    
                   
            if(pyMax < img.shape[1] and pxMax < img.shape[0] and img[pxMax, pyMax] == 0):
                    prox.append( (pxMax, pyMax, state) )
                    
            if(pyMin >= 0 and pxMax < img.shape[0] and img[pxMax, pyMin] == 0):
                    prox.append( (pxMax, pyMin, state) )
                    
            if(pyMin >= 0 and pxMin >= 0 and img[pxMin, pyMin] == 0):
                    prox.append( (pxMin, pyMin, state) )
                    
            if(pyMax < img.shape[1] and pxMin >= 0 and img[pxMin, pyMax] == 0):
                    prox.append( (pxMin, pyMax, state) )
            
             
                
            visitados.add(atual)
            
    return img



def getLine(img):
    line = []
    maior = -1 

    rows, cols, channels = img.shape

    for i in range(cols):
        for j in range(rows):
            if img[j, i] > maior:
                maior = img[j, i]
                px = i
                py = j

    if maior <= 1:
        return -1

    line.append((maior, px, py))

    
    while maior > 0:

        pyMin = py - 1
        pyMax = py + 1
        pxMin = px - 1
        pxMax = px + 1

        
        if pyMin >= 0 and (img[pyMin, px] == (maior - 1) or  img[pyMin, px] == -2):
            maior = (maior - 1)
            if img[pyMin, px] == -2:
                maior = -2
                
            line.append((maior, px, pyMin))
            py = pyMin
            continue

        if pyMax < img.shape[0] and (img[pyMax, px] == (maior - 1) or  img[pyMax, px] == -2) :
            maior = (maior - 1)
            if img[pyMax, px] == -2:
                maior = -2

            line.append((maior, px, pyMax))
            py = pyMax
            continue

        if pxMin >= 0 and (img[py, pxMin] == (maior - 1) or  img[py, pxMin] == -2):
            maior = (maior - 1)
            if img[py, pxMin] == -2:
                maior = -2

            line.append((maior, pxMin, py))
            px = pxMin
            continue

        if pxMax < img.shape[1] and (img[py, pxMax] == (maior - 1) or  img[py, pxMax] == -2):
            maior = (maior - 1)
            if img[py, pxMax] == -2:
                maior = -2

            line.append((maior, pxMax, py))
            px = pxMax
            continue
            
            
            
        if pyMax < img.shape[1] and pxMax < img.shape[0] and (img[pyMax, pxMax] == (maior - 1) or  img[pyMax, pxMax] == -2):
            maior = (maior - 1)
            if img[pyMax, pxMax] == -2:
                maior = -2
                
            line.append((maior, pxMax, pyMax))
            py = pyMax
            px = pxMax
            continue

        if pyMin >= 0 and pxMax < img.shape[0] and (img[pyMin, pxMax] == (maior - 1) or  img[pyMin, pxMax] == -2) :
            maior = (maior - 1)
            if img[pyMin, pxMax] == -2:
                maior = -2

            line.append((maior, pxMax, pyMin))
            py = pyMin
            px = pxMax
            continue

        if pyMin >= 0 and pxMin >= 0 and (img[pyMin, pxMin] == (maior - 1) or  img[pyMin, pxMin] == -2):
            maior = (maior - 1)
            if img[pyMin, pxMin] == -2:
                maior = -2

            line.append((maior, pxMin, pyMin))
            py = pyMin
            px = pxMin
            continue

        if pyMax < img.shape[1] and pxMin >= 0 and (img[pyMax, pxMin] == (maior - 1) or  img[pyMax, pxMin] == -2):
            maior = (maior - 1)
            if img[pyMax, pxMin] == -2:
                maior = -2

            line.append((maior, pxMin, pyMax))
            py = pyMax
            px = pxMin
            continue
        
        

        break

    
    return line



def getLines(img, tamMinLines=10):
    lines = []
    
    while True:
        line = getLine(img)

        if line == -1:
            break 
        
        if len(line) > tamMinLines:
            lines.append( line )

        for coord in line:
            valor, px, py = coord
            if valor >= 2:
                img[py, px] = -2    
          
    return img, lines    


## Função que gera o arquivo de saída

> Essa função itera por todos os tiles no diretório de entrada, os processando com a biblioteca do Skeletonize e então realizando a busca em largura, para depois converter a resposta da busca (que é em px e py referente ao tile) para um par de coordenadas geográficas, e por fim salvar essas coordenadas em um arquivo csv no diretório de saída.

In [4]:

path = 'input'
path_out = 'output'

for file in listdir(path):
    if isfile(join(path, file)):

        f = file
        img = cv2.imread(join(path, f), cv2.IMREAD_COLOR)

        
        #Cortando a imagem
        img = img[0:256, 256:512, 0:1]
        
        image = invert(img)

        # perform skeletonization
        image = skeletonize(image)

        image = invert(image)

        img = image.copy().astype(np.int32)

        img[img[:, :,0] == 255] = -1
        
        i ,lines = findLine(img)

        img[img[:, :,0] != -1] = 0
        img[img[:, :,0] == -1] = 255
        
        j = 1

        linesCoord = []
        for line in lines:
            l = []
            for c in line:
                val, px, py = c

                x = int(file[2:7])
                y = int(file[8:13])
                z = int(file[14:16])

                coord = pixToCoord(x, y, z, 256, 256, px, py)

                l.append(coord)
            
            linesCoord.append(l)
        
        f = file[:16] + '.csv'
        df = DataFrame(linesCoord)

        df.to_csv(join(path_out, f))
